# Google Hash Code 2021


### Author: Alex Delruelle


## Cases:

      Case | Duration  | #Intersections | #Streets |  #Cars  | #Bonus Points
      ----------------------------------------------------------------------
    
         A |       6   |          4     |       5  |      2  |       1000
       
         B |    5070   |       7073     |    9102  |   1000  |       1000
       
         C |    1640   |      10000     |   35030  |   1000  |        100
       
         D |    8071   |       8000     |   95928  |   1000  |       1000
       
         E |     676   |        500     |     998  |   1000  |        500
       
         F |    1992   |       1662     |   10000  |   1000  |        500
       
    Kaggle |    7071   |       8000     |   63968  |   1000  |       1000
   

Google Colab:

   See: https://www.kaggle.com/general/74235
   
   - Activate internet
   - +Add Data (from competition data sets)
   
   ! pip install google-colab
   
   import os
   print(os.listdir("../input/hashcode-2021-oqr-extension"))
   
   Simulation("../input/hashcode-2021-oqr-extension/hashcode.in")

## Code

### Base datastructures

In [1]:
class TrafficLight:
    #
    
    def __init__(self):
        self.Green = False
        self.CarQueue = []       # queue of cars waiting at the traffic light
        self.CarQueueNew = []    # temporary buffer to keep new situation after update
        self.LongestQ = 0        # Keep track how long the queue becomes during the simulation
        
    def SetGreen(self):
        self.Green = True
        
    def SetRed(self):
        self.Green = False
        
    def HasGreen(self):
        # Does this street vave green light on this intersection?
        return(self.Green)
    
    def AddCar(self, Car):
        # Add a car at the end of the CarQueue.
        #print("Added Car: ", Car.ID)
        self.CarQueueNew.append(Car)
        self.LongestQ = max(self.LongestQ, len(self.CarQueueNew))
        
    def FifoPopCar(self):
        # Remove the first car in the CarQueue and return the popped value.
        #print("before POP: ", self.CarQueue[0].ID)
        if self.CarQueue:
            #print(self.CarQueueNew[0].ID)
            return(self.CarQueueNew.pop(0))
        else:
            return(None)
    
    def FirstCar(self):
        # First car in the Queue
        if self.CarQueue: # check if there is a car in the queue
            return(self.CarQueue[0])
        else:
            return(None)
        
    def CarInQueue(self, Car):
        # Is Car in the Queue ?
        return(Car in self.CarQueue)
    
    def QueueLength(self):
        # Return the number of cars in the Queue at the traffic light
        return(len(self.CarQueue))
            
    def Queue(self):
        # Return cars waiting in the queue at the traffic light at this moment in a string
        carlist = ""
        for c in self.CarQueue:
            carlist += str(c.ID) + ", "
        return(carlist)
    
    def SyncQueueAfter(self):
        # Update new situation
        #self.CarQueue = self.CarQueueNew.copy()
        self.CarQueue = self.CarQueueNew[:]
        #print("CarQueue ", self.CarQueue)
        
    def DisplayQueue(self):
        # Show cars waiting in the queue at the traffic light at this moment
        carlist = ""
        for c in self.CarQueue:
            carlist += str(c.ID) + ", "
        print( "Cars in Queue : ", carlist)
    
    def Reset(self):
        # Re-initialise
        self.__init__()


In [2]:
class Street:
    # Each street is unidirectional, with a traffic light at the end.
    # Each street begins at an intersection and ends at another intersection
    # At the traffic light a queue of cars is waiting for green light.
    
    def __init__(self, Name, Begin, End, Length):
        self.Name = Name                   # name of the street
        self.Begin = Begin                 # intersection at the begin of the street
        self.End = End                     # intersection at the end of the street
        self.Length = Length               # lenght of the street, expressed in clockticks to traverse
        self.StopLight = TrafficLight()    # Traffic light at the end of the street
        self.NbrCars = 0                   # Nbr of cars that have this street on their route
        
    def Reset(self):
        # Re-initialise       
        self.StopLight.Reset()


In [3]:
class Schedule:
    # Sequence(list) of tuples (list = (Street, #Clock ticks) ) of subsequent green light.
    # Each street at the intersection can appear only once per schedule.
    # The schedule is repeated continously till the end of the simulation.
    
    def __init__(self):
        self.Schedule = []    # list of tuplelists
        self.Complete = False # becomes True when all streets in the intersection have been added
        self.Pointer = [0,0]  # index in Schedule. First index indicates Schedule tuple, 2nd time in that street
        self.PreviousSchedule = []
        
    def UpdateSchedule(self, street, streetList):
        # Creation of the schedule
        # If street is already green, add 1 clock tick to tuple, otherwise add new tuple in list
        # Called after calling ScheduleComplete(), so building up schedule is not yet finished
        
        if self.Schedule:
            if self.Schedule[-1][0] == street: # already has green, extend schedule
                self.Schedule[-1][1] += 1
            else:
                self.Schedule.append([street,1])
        else:
            # not yet initialised - add first tuple
            self.Schedule.append([street,1])
        
        # Check if Schedule is complete:
        Flag = True
        for s in streetlist:
            Flag = Flag and StreetInSchedule(s)
        self.Complete = Flag
        if self.Complete:
            self.Pointer[0] = len(self.Schedule)   # now at last street in Schedule
            self.Pointer[1] = self.Schedule[-1][1] # at last moment in Schedule
        
            
    def ScheduleList(self):
        # Return the traffic light schedule from the beginning till this moment in native format.
        return(self.Schedule)

       
    def ScheduleComplete(self):
        return(self.Complete)


    def CurrentStreet(self):
        # Current Street in schedule at this moment
        if self.Schedule:
            if self.ScheduleComplete():
                if self.Pointer[1] < self.Schedule[self.Pointer[0]][1]: # still time to spend on current street
                    self.Pointer[1] += 1
                else:
                    if self.Pointer[0] < (len(self.Schedule) - 1):
                        self.Pointer[0] += 1  # move to next tuple in schedule                       
                    else:
                        self.Pointer[0] = 0 # start again at begin of Schedule
                    self.Pointer[1] = 1
                return(self.Schedule[self.Pointer[0]][0])
            else:
                return(self.Schedule[-1][0])
        else:
            return(None)

        
    def StreetInSchedule(self, street):
        # Verify if street is already in the schedule list.
        Flag = False
        for t in self.Schedule:
            if t[0] == street:
                Flag = True
                break
        
        return(Flag)

            
    def ShowSchedule(self):
        # Return the traffic light schedule from the beginning till this moment in string format.
                
        out = ": ["
        for s in self.Schedule:
            out += "(" + s[0].Name + ", " + str(s[1]) + "),"
        out += "]"
        return(out)
    
    def KeepOldSchedule(self):
        # In subsequent tries keep previous schedule
        self.PreviousSchedule = self.Schedule[:]
        
    def KeepBestSchedule(self):
        # Called when previous schedule has a better score than last schedule
        self.Schedule = self.PreviousSchedule[:]
        
    def EqualSchedules(self):
        # return True is the Schedule and PreviousSchedule are identical else False
        Flag = True
        for i in range(len(self.Schedule)):
            Flag &= self.Schedule[i][0] == self.PreviousSchedule[i][0]
            Flag &= self.Schedule[i][1] == self.PreviousSchedule[i][1]
            
        return(Flag)

    
    def Reset(self):
        # Re-initialise
        #self.__init__()
        self.Pointer = [0,0]


In [4]:
list1 = [1, 2, 3, 4, 5]
list2 = [6, 7, 8, 9, 10]
list3 = list1
print(list3)
list1.append(11)
print(list3)

[1, 2, 3, 4, 5]
[1, 2, 3, 4, 5, 11]


In [5]:
list1 = [1, 2, 3, 4, 5]
list2 = [6, 7, 8, 9, 10]
list3 = list1[:]
print(list3)
list1.append(11)
print(list3)

[1, 2, 3, 4, 5]
[1, 2, 3, 4, 5]


In [6]:
list2 = list1[:]
print(list2)
list1.append(11)
print(list2)

[1, 2, 3, 4, 5, 11]
[1, 2, 3, 4, 5, 11]


In [4]:
class Intersection:
    # Crossing of multiple roads
    # Several unidirectional roads can give access to an intersection.
    # At any given time (clock tick) only one road can have access (green light) to the intersection.
    
    def __init__(self, ID):
        self.ID = ID                    # ID number of the intersection
        self.StreetList = []            # All the streets with access to the intersection
        self.TimeTable = Schedule()     # list of tuples/lists (Street, #Clock ticks) of subsequent green light.
                                        # If the list is empty, no roadlight ever got green
        
        # Intersection street selection Methods Names: 
        self.LongestQueue = 1 
        self.FastestCar = 2
        self.Combined = 3
        self.Ratio = 4
                  
    def AddStreet(self, street):
        # StreetList construction
        self.StreetList.append(street)
        
    def AllStreets(self):
        # return list with all streets giving way on intersection
        return(self.StreetList)
    
    def Update(self, street):
        # make sure street has green, other streets on the intersection have red and update schedule

        street.StopLight.SetGreen()
        # Make sure other Stoplights are on red:
        #print(type(self.StreetList), ": ", self.StreetList)
        for s in self.StreetList:
            #print(type(s), ": ", s)
            if s != street:
                s.StopLight.SetRed()
            
    def DisplayIntersection(self):
        # Show the traffic light schedule from the beginning till this moment in string format.                
        print(str(self.ID), ": ", self.TimeTable.ShowSchedule())


    def GreenOnOneStreet(self, Method):   
        # From the streets at an intersection select one along Method and put green light on.
        # Currently only local optimalisation is supported
        #
        # In all cases, if there is only one street giving in on the intersection,
        # it's Traffic Light is put on green for the entire duration of the simulation.
        #
        # Method: from all streets giving in on the intersection, select:
        #  LongestQueue: always the one with the longest car queue 
        #  FastestCar: always the one in which the first car is closest to its destination
        #  Combined: a combination of the previous, i.e. for the last choice, pick LongestQueue, the ones before,
        #            FastestCar. Combined is only used when going for the restriction/limitation in
        #            the Google Hash Code 2021 Problem Statement, i.e. : 
        #                      "And each street name can only be listed once per schedule."

        def SelectLongestQueue(StreetList):
            # return street with longest queue
            
            Choices = []
            for street in StreetList:
                     Choices.append(street.StopLight.QueueLength())
            Longest = np.argmax(Choices)
            #if len(Longest) > 1: # multiple longest Queues, select the one with FirstCar closest to destination
                # these may be all zero:
            #    if Longest[0] == 0: # all streets have an empty queue
            #        s = StreetList[Longest[0]]
            #    else:
            #        Cars = []
            #        for i in Longest:
            #            Cars.append(StreetList[i].StopLight.FirstCar().TimeToDestination())
            #            First = np.argmin(Cars)
            #            s = StreetList[Longest[First[0]]]                        
            #else:
            #    s = StreetList[Longest[0]]
            if StreetList[Longest]:
                s = StreetList[Longest] # There is a street with a queue
            else:
                s = None # no street has a queue
            
            return(s)
        
        def SelectFirstCar(StreetList):
            # return street with Car closest to destination
            
            Flag = False
            Choices = []            
            for street in StreetList:
                #print(street.Name)
                #street.StopLight.DisplayQueue()
                if street.StopLight.QueueLength(): # Check if there is a queue: no queue, no cars
                    Choices.append(street.StopLight.FirstCar().TimeToDestination())
                    Flag = True
                else:
                    Choices.append(np.nan)
            if Flag: # at least 1 found
                First = np.nanargmin(Choices) # returns only the first index
                s = StreetList[First]
                #print(s.Name)
            else:
                s = None
                #print("None Found")
            #if len(First) > 1: # multiple First cars, select the one with longest queue behind it
                    # these may be all zero:
                    #if First[0] == 0: # all streets have an empty queue
                        #s = StreetList[Longest[0]]
                    #else:
                        #Queues = []
                        #for f in First:
                             #Queues.append(StreetList[f].StopLight.QueueLength())
                        #Longest = np.argmax(Queues)
                        #s = StreetList[First[Longest[0]]]                
            #else:
                #s = StreetList[First[0]]
            
            
            return(s)
        
        def CheckIfAlreadyInSchedule(street):
            return(self.TimeTable.StreetInSchedule(street))
        
        ListOfStreets = self.AllStreets()
        s = None
        
        # Make sure intersection has a street
        if len(ListOfStreets) == 0:
            print("Intersection ", Crossing.ID, "has no street")
            sys.exit()
        
        # For intersections with only 1 street, select that one, and put traffic light on green permanently:
        if len(ListOfStreets) == 1:           
            s = ListOfStreets[0]
        else:            
            if Method == self.LongestQueue:
                # put the one with the longest queue on green. If longest queues have equal length, take the Fastestcar                   
                s = SelectLongestQueue(ListOfStreets)
            elif Method == self.FastestCar:
                # put the one with the car closest to destination on green. If more than one choice, take longest queue
                s = SelectFirstCar(ListOfStreets)
            elif Method == self.Combined:
                if  self.TimeTable.CurrentStreet():
                    # if there is already a street with green, and it has a queue, stay on that one:               
                    if  self.TimeTable.CurrentStreet().StopLight.HasGreen() \
                        and self.TimeTable.CurrentStreet().StopLight.QueueLength():
                        s = self.TimeTable.CurrentStreet()
                    # check if all streets have already been selected, if so, stay on last street:
                    elif len(self.TimeTable.ScheduleList()) == len(ListOfStreets):
                        s = self.TimeTable.CurrentStreet()
                    # only one traffic light switch left, take the one with the longest queue:
                    elif len(self.TimeTable.ScheduleList()) == (len(ListOfStreets) - 1):
                        s = SelectLongestQueue(ListOfStreets)
                        # Check if it was already in the list
                        if CheckIfAlreadyInSchedule(s):  # already used, should take 2nd longest queue
                            s = SelectFirstCar(ListOfStreets)
                            if CheckIfAlreadyInSchedule(s):
                                s = None
                    else:
                        s = SelectFirstCar(ListOfStreets)
                        if CheckIfAlreadyInSchedule(s):
                            s = SelectLongestQueue(ListOfStreets)
                            if CheckIfAlreadyInSchedule(s):
                                s = None
                        
                else: # no street selected yet, take the one with the first car closest to destination
                    s = SelectFirstCar(ListOfStreets)
                    if CheckIfAlreadyInSchedule(s):
                        s = SelectLongestQueue(ListOfStreets)
                        if CheckIfAlreadyInSchedule(s):
                            s = None
            elif Method == self.Ratio:
                # Schedules are already set:
                s = self.TimeTable.CurrentStreet()
                
        # Adapt Intersection Schedule and Stoplight situation:
        #if s:
        #    self.Update(s)

        return(s)

            
    def Reset(self):
        # Re-initialise
        self.TimeTable.Reset()
       

In [8]:
ll = [0,1,2,3,4]
current = 0
for s in range(current+1, len(ll)):
   print(s)

1
2
3
4


In [5]:
class Route:
    # Sequence of streets and position in that sequence.
    
    def __init__(self, RouteList):
        self.StreetList = RouteList    # List of streets
        self.RoutePosition = [0,1]     # Position in the route. 1st coordinate is the index in the route list, i.e. in 
                                       # which street is the car. 2nd coordinate is the position in the street, counting
                                       # down, i.e. full lenght if at begin of the street, 1 if at the traffic light.
                                       # The initial position of a car in the first street is at the end.
    def AtTrafficLight(self):
        return(self.RoutePosition[1] == 1)
    
    def LastStreetInRoute(self):
        return((self.RoutePosition[0]+1) == (len(self.StreetList)))
    
    def AtEndOfRoute(self):
        return( self.LastStreetInRoute() and self.AtTrafficLight() )    
    
    def CurrentStreet(self):
        return(self.StreetList[self.RoutePosition[0]])
        
    def NextStreet(self):
        if self.LastStreetInRoute(): # Current street is last in route
            return(None)
        else:
            return(self.StreetList[self.RoutePosition[0]+1])
        
    def PositionInCurrentStreet(self):
        return(self.RoutePosition[1])
        
    def Remainder(self):
        # Remaining distance in route
        Count = 0
        if self.LastStreetInRoute():
            Count = self.RoutePosition[1] - 1 # remainder of last street
        else:
            Count = self.RoutePosition[1] - 1 # remainder of current street
            # loop through rest of the route
            for i in range(self.RoutePosition[0] + 1, len(self.StreetList)):
                Count += self.StreetList[i].Length
                
        return(Count)
                   
    def Move1(self):
        # Advance one position in Route
        if not self.AtEndOfRoute():
            if self.AtTrafficLight():
                self.RoutePosition[0] += 1
                self.RoutePosition[1] = self.StreetList[self.RoutePosition[0]].Length  # Count down from Length to 1
            else: 
                # Continue on current street
                self.RoutePosition[1] -= 1
            
    def Reset(self):
        # Re-initialise
        self.RoutePosition = [0,1]


In [6]:
class Car:
    # Properties of a car
    # A car can have 3 statuses: "on a road" or "on a road, in a queue before a traffic light" or "arrived at destination"
    # When "arrived at destination", The RoutePosition is put on -1 and the ArrivalTime is > 0.
    
    def __init__(self, ID, RouteList):
        self.ID = ID                    # Car ID
        self.Path = Route(RouteList)    # Route object
        self.ArrivalTime = 0            # Clocktick at which the car arrives at destination, 0 indicates not yet arrived
        self.History = []               # Progress record: subsequent streets + time spend on that street.
        
    def NewPosition(self, time):
        # If at destination: store ArrivalTime; else advance one position.
        
        def CheckEndOf(car):
            if car.Path.AtEndOfRoute():                
                #print("Car ", car.ID, "has arrived at end of route")
                car.ArrivalTime = time
            elif car.Path.AtTrafficLight():
                car.Path.CurrentStreet().StopLight.AddCar(car)
        
        if self.Path.AtTrafficLight(): # did already reach the traffic light in the previous clocktick
            if self.Path.CurrentStreet().StopLight.HasGreen():
                if self == self.Path.CurrentStreet().StopLight.FirstCar(): # is the first in the queue
                    Car = self.Path.CurrentStreet().StopLight.FifoPopCar() # remove car from Queue
                    if Car != self:                                                   # <=== shorten this
                        print("Wrong car popped from list. Should be: ", self.ID)
                        if Car:
                            print("is:", Car.ID)
                        else:
                            print("is: None") 
                    self.Path.Move1()
                    CheckEndOf(self)
        else:
            self.Path.Move1()
            CheckEndOf(self)
        
        self.AddHistory(self.Path.CurrentStreet())
    
        return(None)
           
    def TimeToDestination(self):
        # Fastest possible time to the cars destination, i.e. with all traffic lights on its route on green.
        if self.ArrivalTime:
            Count = 0
        else:
            Count = self.Path.Remainder()
           
        return(Count)
            
    def Arrived(self):
        # if already arrived at destination returns Arrival moment, else returns 0.
        return(self.ArrivalTime)

    def DisplayCar(self):
        # Show status of car at this moment:
        if (self.ArrivalTime):
            print(self.ID, " : Arrived at: ", self.ArrivalTime)
        else:            
            print(self.ID, " : Minimal time remaining = ", self.TimeToDestination(), ", Position (", \
               self.Path.PositionInCurrentStreet(), ") in ", self.Path.CurrentStreet().Name)       
        for l in range(len(self.History)):
            print("\t[", self.History[l][0].Name, ":", self.History[l][1], "s]")
    
    def AddHistory(self, street):
        # add 1 second to the Car's History in street
        if self.History[-1][0] == street: 
            self.History[-1][1] += 1   # if still in the same street, increase 1
        else:
            self.History.append([street, 1]) # add a new street
    
    def Reset(self):
        # Re-initialise
        self.Path.Reset()
        self.Path.CurrentStreet().StopLight.AddCar(self)
        self.ArrivalTime = 0
        h = self.History[0][0]
        self.History = []
        self.History.append([h, 1])
          

In [7]:
class Clock:
    # Clock used to coordinate the simulation
    # Actually a chronometer, starts at 0, ticks till Duration is reached.
    
    def __init__(self, Duration):
        self.Duration = Duration        # total duration of the simulation in clock ticks
        self.TimeNow = 0                # clock time, i.e. the time at current simulation tick
        
    def Tick(self):
        # As long as the chronometer is still ticking, return True
        self.TimeNow += 1
        return(self.TimeNow <= self.Duration)
        
    def Now(self):
        # The time at this moment
        return(self.TimeNow)
    
    def Reset(self):
        # Re-initialise
        self.TimeNow = 0

### I/O routines

In [8]:
def ReadInputFile(FileName):
    # Read input
    
    Streets = []
    Cars = []
    
    with open(FileName, "r") as file:
        lines = file.readlines()
        D, I, S, V, F = lines[0].strip().split(" ")
        
        for n in range(1, int(S) + 1):
            Streets.append(lines[n].replace("\n", "").split(" "))
        for n in range(int(S) + 1, int(S) + 1 + int(V)):
            Cars.append(lines[n].replace("\n", "").split(" "))
            
    return(D, I, S, V, F, Streets, Cars)

In [9]:
def PrintInputFile(D, I, S, V, F, Streets, Cars):
    # Show all input parameters.
            
    print(f"Duration = {D} s, {I} intersections, {S} streets, {V} cars, {F} bonus points\n")
    
    print("Streets :\n")
    for n in range(len(Streets)):
        print(f"B = {Streets[n][0]}, E = {Streets[n][1]}, {Streets[n][2]}, time needed = {Streets[n][3]}\n")
    
    print("Cars :\n")
    for n in range(len(Cars)):
        print(f"P (number of streets to travel) = {Cars[n][0]}: {Cars[n][1:]}\n" )
    
    return(None)

In [10]:
def PrintSubmission(IntersectionList, FileName = None):
    # File format required for the Google Hash code 2021 submission
    # If FileName given, will to write to that file, otherwise on screen display.
    #
    # A : ( 0 ≤ A ≤ I ) the number of intersections for which you specify the schedule.
    # For each intersection
    # i : the first line containing a single integer i (0 ≤ i < I ) – the ID of the intersection
    # E : list of green lights [street name, T (1 ≤ T ≤ D ) for how long each street will have a green light, ]
    
    if FileName:
        original_stdout = sys.stdout # Save a reference to the original standard output
        f = open(FileName, 'w')
        sys.stdout = f # Change the standard output to the file we created.
        
    # A: number of intersections:
    Count = 0
    for n in IntersectionList:    # loop through all intersections
        if n.TimeTable.ScheduleList(): # skip intersection if there is no schedule
            Count +=1
    print(Count)                                        # A: number of intersections
 
    #for n in range(len(IntersectionList) ):    # loop through all intersections
        #print(IntersectionList[n].ID)                  # i: intersection ID
        #print(len(IntersectionList[n].StreetList))     # E(i): number of streets for this instersection
        #for s in IntersectionList[n].StreetList:       # schedule =
        #    print(s[0].Name, " ", s[1])                # list of subsequent (StreetName, duration) green light 
                                                        # for this intersection
    for n in IntersectionList:    # loop through all intersections
        if n.TimeTable.ScheduleList(): # skip intersection if there is no schedule
            print(n.ID)                  # i: intersection ID
            print(len(n.TimeTable.ScheduleList()))     # E(i): number of streets for this intersection
            for s in n.TimeTable.ScheduleList():
                print(f"{s[0].Name} {s[1]}")
        
    if FileName:
            sys.stdout = original_stdout # Reset the standard output to its original value
            
    return(None)

SimE.Intersections[1].TimeTable.ScheduleList()[0]

### Simulation routines

In [2]:
import math
A = [12, 24, 27, 30, 36]
print(math.gcd(*A))

3


In [11]:
import math

def listGCD(NumberList):
    # Calculate the GCD of a list, 2 or more numbers.

    result = NumberList[0]
    for i in NumberList[1:]:
        result = math.gcd(result, i)
        
    return(result)

In [3]:
print(listGCD(A))

3


In [12]:
def TransformFractions(FractionList):
    # Convert a list of decimal fractions into a list of integers and normalize using greatest common divisor
    
    fraction = 0.1
    m = len(FractionList)
    Flag = True
    while Flag:        
        for i in range(m):
            if FractionList[i] < fraction:
                fraction /= 10
                break
        Flag = False
    # All Fractions are larger than fraction
    NewFractionList = []
    for i in range(m):
        NewFractionList.append(round(FractionList[i]/fraction))
    # Normalize to smallest numbers by dividing by GCD
    #GCD = math.gcd(*NewFractionList) # works only in python 3.9, not yet available on google colab
    GCD = listGCD(NewFractionList)
    if GCD > 0:
        NewFractionList = [int(x/GCD) for x in NewFractionList]
    
    return(NewFractionList)        

In [17]:
TransformFractions([0.222, 0.07, 0.33333, 0.5555])

[22, 7, 33, 56]

In [18]:
l = TransformFractions([0.222, 0.07, 0.33333, 0.5555])
l.sort(reverse=True)
l

[56, 33, 22, 7]

In [19]:
TransformFractions([0.9, 0.03, 0.33333, 0.6])

[30, 1, 11, 20]

In [20]:
TransformFractions([0.5, 0.5])

[1, 1]

In [13]:
def Sort2ListsInSync(List1, List2):
    # Sort List1 and List2 based on List2
   
    # initializing blank dictionary
    Dict = {} 
     
    # initializing blank list
    l1 = []
    l2 = []
     
    # Addition of two list in one dictionary
    Dict = {List1[i]: List2[i] for i in range(len(List2))} 
     
    # sorting of dictionary based on value
    sDict = {k: v for k, v in sorted(Dict.items(), reverse=True, key=lambda item: item[1])}
     
    # Element addition in the list
    for k,v in sDict.items():
        l1.append(k)
        l2.append(v)        
    
    return(l1, l2)

In [22]:
list1 = ["a", "b", "c", "d", "e", "f", "g", "h", "i"]
list2 = [ 0,   1,   1,    0,   1,   2,   2,   0,   1]
Sort2ListsInSync(list1,list2)

(['f', 'g', 'b', 'c', 'e', 'i', 'a', 'd', 'h'], [2, 2, 1, 1, 1, 1, 0, 0, 0])

In [39]:
import time

def procedure():
    time.sleep(2.5)
    print("Done Something")

# measure process time
#t0 = time.time()()
#procedure()
#print (time.time()(), "seconds process time")

# measure wall time
t0 = time.time()
procedure()
print (time.time() - t0, "seconds wall time")

Done Something
2.501817464828491 seconds wall time


In [14]:
import numpy as np
import sys
import time

MaxRunTime = 32400  # in seconds  =  CPU Notebook <= 9 hours run-time

class Simulation:
    # A simlation is determined by a duration in time, a set of streets, a set of intersections and a number of cars.
    # The result is calculate via a score function using a fixed and a variable bonus depending on remaining time. 
    
    def __init__(self, InputFile):
        
        # Get Input        
        self.D, self.I, self.S, self.V, self.F, self.StreetList, self.CarList = ReadInputFile(InputFile)

        # Constants:
        
        # Set Bonus Value:
        self.Bonus = int(self.F)   
        
        # Variables:
   
        self.Timer = Clock(int(self.D)) # Create a Timer
        self.Intersections = []         # List of Intersections
        self.Streets = {}               # Dictionary of Streets
        self.Cars = []                  # List of Cars
        self.ScoreList = []
        
        # Create the list of Intersections (at this point we only know how many intersections):
        for i in range(int(self.I)):
            self.Intersections.append(Intersection(i))

        # Create a dictionary of Streets and update Intersections:
        for s in range(int(self.S)):
            self.Streets[self.StreetList[s][2]] = Street(self.StreetList[s][2], \
                                                         int(self.StreetList[s][0]), int(self.StreetList[s][1]), \
                                                         int(self.StreetList[s][3]))
            self.Intersections[int(self.StreetList[s][1])].AddStreet(self.Streets[self.StreetList[s][2]])
        
        # Create a list of Cars and update StreetQueues:
        for c in range(int(self.V)):
            # Convert route streetnamelist into StreetObjectList:
            StreetObjList = []
            for s in self.CarList[c][1:]:
                StreetObjList.append(self.Streets[s])
                self.Streets[s].NbrCars += 1
            # now the Cars list:
            self.Cars.append(Car(c, StreetObjList))
            self.Streets[self.CarList[c][1]].StopLight.AddCar(self.Cars[c]) # initial position of every car is at the traffic light of the first street in it's route.
            self.Cars[c].History.append([StreetObjList[0], 0]) # First street in the Car's history, Put at 0, because at traffic light at the start.
            
        # Sync all traffic lights:
        for s,st in self.Streets.items():
            st.StopLight.SyncQueueAfter()
            
        # Initialise intersection schedule based on traffic ratios, longest queue and Car closest to destination:        
        self.InitialiseSchedules()
        

    def Score(self):
        # "Cost" function. Calculate total score at the end of the simulation
    
        score = 0    
        for c in self.Cars:
            if c.Arrived():
                score += self.Bonus       # add bonus
                score += self.Timer.Duration - c.Arrived()   # one point for each second left when the car finished its route.                
        return(score)
    
    def NbrCarsArrived(self):
        # How many cars have arrived at this moment?
        
        Count = 0
        for car in self.Cars:
            if car.Arrived():
                Count += 1
        return(Count)

    def DisplayIntersections(self):
        # Show the intersections:
        
        print("\nIntersection Schedules :\n------------------------")
        for i in self.Intersections:
            i.DisplayIntersection()
    
    def Display(self):
        # Show a snapshot of the simulation at this moment.
    
        # The streets:
        print("\nStreets - Cars in Traffic Light Queue :\n---------------------------------------")
        for s, v in self.Streets.items():
            if v.StopLight.HasGreen():
                Color = 'Green'
            else:
                Color = 'Red'
            print(s + ": " + Color + ": " + v.StopLight.Queue())
    
        # The cars:
        print(self.NbrCarsArrived(), "cars out of", len(self.Cars), "have arrived.")
        print("\nCars :\n------")
        for c in self.Cars:
            c.DisplayCar()

    def RunSimulation(self, FileName = None, Log = False):
        # Execute all steps in the simulaton
        # If FileName given, will write a log to that file, otherwise on-screen display.
        # Simulaton logs quickly become very bulky, e.g. case E generates 117Mb of text.
        # Jupyter Notebooks doesn't handle bulky output well. Beter write it in a file and consult via a text editor.
        #
        # Traffic light queues:
        # As the checking happens in a serial mode, we have to work with a before and after list.
        # If we work with one list and remove a car, when we check a following car it could be that it is first in list
        # because the original first car was removed. Only one car per tick is allowed to move forward at a Traffic Light.
        
    
        if FileName:
            original_stdout = sys.stdout # Save a reference to the original standard output
            f = open(FileName, 'w')
            sys.stdout = f # Change the standard output to the file we created.
        
        if Log:
                PrintInputFile(self.D, self.I, self.S, self.V, self.F, self.StreetList, self.CarList)
                
        # start chronometer:
        StartTime = time.time()
        
        FullCycleCount = 0
        while True:
            # check chronometer:
            if (time.time() - StartTime) > MaxRunTime:
                break
            
            # Re-initialise for re-run:
            self.Reset()
            
            FullCycleCount += 1
                    
            if Log:      
                print("\nFullCycle", FullCycleCount, "\n=============\nInitial situation :\n===================")
                self.DisplayIntersections()
                self.Display() # initial situation
    
            while self.Timer.Tick():
                if Log:
                    print("\nClockTick: ", self.Timer.Now(), "\n==========")

                # On all intersections put 1 traffic light on green
                for i in self.Intersections:
                    street = i.GreenOnOneStreet(i.Ratio)
                    i.Update(street)
    
                # Process cars at green light and cars on the move in a street:
                for car in self.Cars:
                    if not car.Arrived():
                        car.NewPosition(self.Timer.Now())
                    #if car.Arrived(): # no need to do anything with an already arrived car
                        #print("Car ", car.ID, " has arrived")
                    
                # Now sync all traffic lights:
                for s,st in self.Streets.items():
                    st.StopLight.SyncQueueAfter()
                    
                if Log:
                    self.Display() # situation after ClockTick
                    
            self.ScoreList.append(self.Score())
            print("\nIntermediate Score = ", self.Score(), "\n==========")
            if len(self.ScoreList) > 1:
                if self.ScoreList[-1] <= self.ScoreList[-2]: # last score is worse then or equal to previous score
                    self.KeepBestSchedules() # restore previous schedule
                    print("Final Score 1= ", self.ScoreList[-2])                     
                    break
                elif not self.OptimiseSchedules():
                    print("Final Score 2= ", self.ScoreList[-1]) 
                    break # if no optimization done, quit.                      
            else:
                if not self.OptimiseSchedules():
                    print("Final Score 3= ", self.ScoreList[-1]) 
                    break # if no optimization done, quit.  
            
        if FileName:
            sys.stdout = original_stdout # Reset the standard output to its original value

        return(None)

    def InitialiseSchedules(self):
        # Called only during initialisation of the simulation
        # 
                        
        for i in reversed(self.Intersections): # do it backwards to allow removing intersections while processing list
            if len(i.StreetList) == 1: # if only one street giving in on intersection, put green for entire duration
                i.TimeTable.Schedule.append([ i.StreetList[0], self.Timer.Duration ])
                #print(i.ID, i.TimeTable.Schedule)
            else: # more than 1 street giving in on the intersection
                # Calculate ratio's per intersection and set schedule:
                Total = 0
                for street in i.StreetList:
                    Total += street.NbrCars
                if Total == 0:
                    print("Intersection", i.ID, "is never used by a car")
                    # remove intersection from the Intersection list
                    self.Intersections.remove(i)
                else:
                    FractionList = []
                    for street in i.StreetList:
                        FractionList.append(street.NbrCars/Total)
                    ScheduleList = TransformFractions(FractionList)
                    # Rearange to prioritise the streets expecting the most cars, i.e. the longest time first:
                    #tl = ScheduleList.sort(reverse=True) # sort from longest to smallest time                
                    sl, tl = Sort2ListsInSync(i.StreetList, ScheduleList)
                    #print(tl, sl)
                    # Rearange subsequent streets with same duration, to favour First car closest to destination
                    for m in range(1, len(tl)):
                        if tl[m] == tl[m-1]:
                            a = sl[m-1].StopLight.FirstCar()
                            b = sl[m].StopLight.FirstCar()
                            if a and b: # there are 2 cars
                                if b.TimeToDestination() < a.TimeToDestination():
                                    # reverse order
                                    h = sl[m]
                                    sl[m] = sl[m-1]
                                    sl[m-1] = h
                            elif a or b:
                                if b: # a is empty, b goes first
                                    h = sl[m]
                                    sl[m] = sl[m-1]
                                    sl[m-1] = h
                            # else there is no a nor b - do nothing
                    # Create schedule:
                    Count = 0
                    for street in sl:
                        i.TimeTable.Schedule.append([ street, tl[Count] ])
                        Count += 1
                    
            i.TimeTable.Complete = True # Schedule is already set
            
        return(None)
    
    
    def OptimiseSchedules(self):
        # Compare and adjust the schedules to the longest queues in the streets.
        # Return True if an optimization attempt is made, False if the schedules remain the same
        
        # Keep record of previous schedule
        for i in self.Intersections:
            i.TimeTable.KeepOldSchedule()
        
        for i in self.Intersections:
            if not len(i.TimeTable.Schedule) == 1:
                NotChanged = True
                for t in i.TimeTable.Schedule:
                    #if LongestQ is larger or smaller than the time set for the street, adjust the time
                    if not(t[0].StopLight.LongestQ == t[1]):                  
                        t[1] = t[0].StopLight.LongestQ
                        NotChanged = False
                if NotChanged:
                    print("Intersection ", i.ID, "No changes due to Queue length")
                    # try interchanging the order in the schedule
                    NewSchedule = []
                    NewSchedule.append(i.TimeTable.Schedule[-1])
                    NewSchedule[1:] = i.TimeTable.Schedule[:-1]
                    i.TimeTable.Schedule = NewSchedule[:]
                    
        # Check for changes in the schedules:
        Equal = True
        for i in self.Intersections:       
            if len(i.TimeTable.Schedule) > 1:
                Equal = Equal and i.TimeTable.EqualSchedules()
        
        if (Equal):
            print("No changes to schedule")
        
        return(not Equal)


    def KeepBestSchedules(self):
        # Restore previous schedules.
        
        for i in self.Intersections:
            i.TimeTable.KeepBestSchedule()
        print('In KeepBestSchedules')
        return(None)


    def Reset(self):
        # Reinitialise data structures for a re-run
        self.Timer.Reset()
        
        for i in self.Intersections:
            i.Reset()
        
        for s in self.Streets.items():
            s[1].Reset()
            
        for c in self.Cars:
            c.Reset()
            
        for s,st in self.Streets.items():
            st.StopLight.SyncQueueAfter()
            #print(st.Name, st.StopLight.CarQueue, st.StopLight.CarQueueNew)
            
        


In [25]:
list1 = [1,2,3]
list2 =[]
list2.append(list1[-1])
list2[1:] = list1[:-1]
print(list2)

[3, 1, 2]


In [26]:
for i in list2:
    if i == 1:
        list2.pop(i)
print(list2)        

[3, 2]


## Perform Simulations

### Case A

#### Read Input

In [27]:
D, I, S, V, F, StreetList, CarList = ReadInputFile("a.txt")

In [28]:
PrintInputFile(D, I, S, V, F, StreetList, CarList)

Duration = 6 s, 4 intersections, 5 streets, 2 cars, 1000 bonus points

Streets :

B = 2, E = 0, rue-de-londres, time needed = 1

B = 0, E = 1, rue-d-amsterdam, time needed = 1

B = 3, E = 1, rue-d-athenes, time needed = 1

B = 2, E = 3, rue-de-rome, time needed = 2

B = 1, E = 2, rue-de-moscou, time needed = 3

Cars :

P (number of streets to travel) = 4: ['rue-de-londres', 'rue-d-amsterdam', 'rue-de-moscou', 'rue-de-rome']

P (number of streets to travel) = 3: ['rue-d-athenes', 'rue-de-moscou', 'rue-de-londres']



#### Simulation

In [29]:
list1 = ["c", "b", "d", "a"]

list2 = [2, 3, 1, 4]


zipped_lists = zip(list1, list2)

sorted_pairs = sorted(zipped_lists)


tuples = zip(*sorted_pairs)

list1, list2 = [ list(tuple) for tuple in  tuples]

In [30]:
list1

['a', 'b', 'c', 'd']

In [31]:
list2

[4, 3, 2, 1]

In [32]:
tuples

In [33]:
l1 = [3, 2, 4, 1, 1]
l2 = ['three', 'two', 'four', 'one', 'second one']
l = zip(*sorted(zip(l1, l2)))
l

In [34]:
print(CarList)

[['4', 'rue-de-londres', 'rue-d-amsterdam', 'rue-de-moscou', 'rue-de-rome'], ['3', 'rue-d-athenes', 'rue-de-moscou', 'rue-de-londres']]


In [15]:
SimA1 = Simulation("a.txt")

In [16]:
SimA1.InitialiseSchedules()

In [17]:
SimA1 = Simulation("a.txt")
SimA1.RunSimulation(FileName = None, Log=True)

Duration = 6 s, 4 intersections, 5 streets, 2 cars, 1000 bonus points

Streets :

B = 2, E = 0, rue-de-londres, time needed = 1

B = 0, E = 1, rue-d-amsterdam, time needed = 1

B = 3, E = 1, rue-d-athenes, time needed = 1

B = 2, E = 3, rue-de-rome, time needed = 2

B = 1, E = 2, rue-de-moscou, time needed = 3

Cars :

P (number of streets to travel) = 4: ['rue-de-londres', 'rue-d-amsterdam', 'rue-de-moscou', 'rue-de-rome']

P (number of streets to travel) = 3: ['rue-d-athenes', 'rue-de-moscou', 'rue-de-londres']


FullCycle 1 
Initial situation :

Intersection Schedules :
------------------------
0 :  : [(rue-de-londres, 6),]
1 :  : [(rue-d-athenes, 1),(rue-d-amsterdam, 1),]
2 :  : [(rue-de-moscou, 6),]
3 :  : [(rue-de-rome, 6),]

Streets - Cars in Traffic Light Queue :
---------------------------------------
rue-de-londres: Red: 0, 
rue-d-amsterdam: Red: 
rue-d-athenes: Red: 1, 
rue-de-rome: Red: 
rue-de-moscou: Red: 
0 cars out of 2 have arrived.

Cars :
------
0  : Minimal time rem

In [ ]:
h = True
h &= SimA1.Intersections[1].TimeTable.EqualSchedules()
h

In [ ]:
h = False
g = True

h &= g
h

In [ ]:
SimA1.OptimiseSchedules()

In [ ]:
SimA1.ScoreList

In [ ]:
SimA1.Intersections[1].TimeTable.ScheduleList()

In [ ]:
SimA1.Intersections[0].TimeTable.Complete

In [ ]:
SimA1.Intersections[1].StreetList[0].StopLight.LongestQ

In [ ]:
for car in SimA1.Cars:
    for street in car.Path.StreetList:
        print(street.NbrCars)

In [ ]:
SimA1.Intersections[1].StreetList[1].NbrCars

In [ ]:
SimA1.Intersections[0].TimeTable.Schedule

In [ ]:
if len(SimA1.Intersections[0].StreetList) == 1:
    print(True)

In [ ]:
SimA1.Cars[0].DisplayCar()

In [ ]:
SimA1.Cars[0].Arrived()

In [ ]:
SimA1.Cars[1].Path.CurrentStreet().StopLight.CarQueueNew

In [ ]:
SimA1.Cars[1].Path.RoutePosition[0]

In [ ]:
SimA1.Score()

In [ ]:
SimA1.Intersections[1].TimeTable.Schedule[0][0].Name

In [ ]:
SimA1.Intersections[1].TimeTable.KeepBestSchedule()

In [ ]:
PrintSubmission(SimA1.Intersections)

In [ ]:
PrintSubmission(SimA1.Intersections, "a.sub")

In [ ]:
from timeit import timeit
SimA2 = Simulation("a.txt")
timeit('SimA2.RunSimulation("a.simlog")', number=1, globals=globals())
SimA2.Score()

In [ ]:
SimA2.Streets['rue-de-londres'].StopLight

In [ ]:
print(SimA1.Cars[1].Path.RoutePosition[0])
print(len(SimA1.Cars[1].Path.StreetList) -1)

In [ ]:
for c in SimA1.Cars:
    print(c.Arrived())

In [ ]:
print(SimA1.Timer.Now())

### Case B

SimB = Simulation("b.txt")
SimB.RunSimulation("b.simlog")

### Case C

SimC = Simulation("c.txt")
SimC.RunSimulation("c.simlog")

### Case D

SimD = Simulation("d.txt")
SimD.RunSimulation("d.simlog")

### Case E

In [ ]:
SimE.Streets["ca-ejj"].StopLight.CarQueue[0].ID

In [ ]:
SimE.Streets["ca-ejj"].StopLight.CarQueueNew[0].ID

In [ ]:
SimE.Cars[275].Path.CurrentStreet().StopLight.CarQueueNew

In [ ]:
SimE.Cars[275].Path.AtEndOfRoute()

In [ ]:
SimE.Cars[278].Path.AtEndOfRoute()

In [ ]:
SimE.Cars[278].Path.CurrentStreet().StopLight.Queue()

In [ ]:
SimE.Cars[278].Path.CurrentStreet().StopLight.CarQueue[0].ID

In [ ]:
SimE.Cars[278].Path.CurrentStreet().StopLight.CarQueueNew[0].ID

In [18]:
from timeit import timeit
SimE = Simulation("e.txt")
timeit('SimE.RunSimulation("e.simlog", Log=False)', number=1, globals=globals())

25.591286457000024

In [19]:
SimE.ScoreList

[678543, 702310, 718478, 718354]

In [ ]:
SimE.Intersections[1].StreetList

In [ ]:
SimE.Intersections[1].StreetList[0].Name

In [ ]:
SimE.Intersections[1].TimeTable.ShowSchedule()

In [ ]:

ArrivalTimes = []
for c in SimE.Cars:
    if c.ArrivalTime == 0:
        ArrivalTimes.append(np.nan)
    else:
        ArrivalTimes.append(c.ArrivalTime)
print(ArrivalTimes)


In [ ]:
l = []
Count = 0
i = 0
end = len(ArrivalTimes)
while i < end:
    try:
        h = ArrivalTimes.index(16, i, end)
        l.append(h)
        i = l[Count]+1
        Count += 1
    except ValueError:
        break    
        
print(l)

In [ ]:
ArrivalTimes[334]

In [ ]:
for s in SimE.Cars[334].Path.StreetList:
    print(s.Name, s.Length)

In [ ]:
SimE.Streets['ejj-eib'].StopLight

In [ ]:
np.nanargmin(ArrivalTimes)

In [ ]:
ArrivalTimes[np.nanargmin(ArrivalTimes)]

In [ ]:
SimE.Cars[278].ArrivalTime

In [ ]:
ArrivalTimes[278]

In [ ]:
for s in SimE.Cars[278].Path.StreetList:
    print(s.Name, s.Length)

In [ ]:
SimE.Cars[278].TimeToDestination()

In [ ]:
SimE.Score()

In [ ]:
SimE.NbrCarsArrived()

In [ ]:
len(SimE.Intersections[499].StreetList)

In [ ]:
PrintSubmission(SimE.Intersections, "e11.sub")

In [ ]:
print(SimE.Timer.Now())
print(len(SimE.Intersections))
print(len(SimE.Streets))
print(len(SimE.Cars))

In [ ]:
SimE.Intersections[121].TimeTable.ScheduleList()

### Case F

SimF = Simulation("f.txt")
timeit('SimF.RunSimulation("f.simlog")', number=1, globals=globals())

SimF.Score()

PrintSubmission(SimF.Intersections, "f.sub")

### Case B

SimB = Simulation("b.txt")
timeit('SimB.RunSimulation(Log=False)', number=1, globals=globals())

SimB.Score()

PrintSubmission(SimB.Intersections, "b4.sub")

### Case C

SimC = Simulation("c.txt")
timeit('SimC.RunSimulation("c.simlog", Log=False)', number=1, globals=globals())

SimC.Score()

PrintSubmission(SimC.Intersections, "c3.sub")

### Case D

SimD4 = Simulation("d.txt")
timeit('SimD4.RunSimulation("d4.simlog", Log=False)', number=1, globals=globals())

SimD4.Score()

PrintSubmission(SimD4.Intersections, "d4.sub")

SimD4.NbrCarsArrived()   

SimE11 = Simulation("e10cars.txt")
timeit('SimE11.RunSimulation("E10cars.simlog", Log=True)', number=1, globals=globals())

SimE11.Score()

PrintSubmission(SimE11.Intersections, "E10cars.sub")

SimE11.NbrCarsArrived()

len(SimE11.Intersections[499].StreetList)

ArrivalTimes = []
for c in SimE11.Cars:
    if c.ArrivalTime == 0:
        ArrivalTimes.append(np.nan)
    else:
        ArrivalTimes.append(c.ArrivalTime)
print(ArrivalTimes)


len(SimE11.Cars[9].Path.StreetList)

SimE11.Cars[7].ArrivalTime

for s in SimE11.Cars[7].Path.StreetList:
    print(s.Name, s.Length)

SimE11.Reset()

q = []
for s in SimE11.Intersections[499].StreetList:
    q.append(s.StopLight.QueueLength())
print(q)

SimE11.Intersections[368].DisplayIntersection()

SimE10.RunSimulation("E11cars.simlog", Log=True)

SimE10.Streets['']

SimK = Simulation("Kaggle/hashcode.in")
SimK.RunSimulation("K.simlog", Log=False)

Intersection 5216 is never used by a car

In [41]:
from timeit import timeit
# SimK = Simulation("../input/hashcode-2021-oqr-extension/hashcode.in")
SimK = Simulation("Kaggle/hashcode.in")
timeit('SimK.RunSimulation("K.simlog", Log=False)', number=1, globals=globals())

Intersection 5216 is never used by a car


KeyboardInterrupt: 

In [ ]:
SimK.ScoreList

In [ ]:
PrintSubmission(SimK.Intersections, "Kaggle.sub")

In [42]:
sys.version

'3.9.1 (default, Dec 11 2020, 14:32:07) \n[GCC 7.3.0]'